In [16]:
import pandas as pd
import gzip
import json

In [17]:
!pip install --no-deps --upgrade pyarrow==14.0.2

In [18]:
!pip install --no-deps xformers trl peft accelerate bitsandbytes

In [4]:
from datasets import load_dataset
data_files = {"cot_fs_noopt_train.jsonl.gz", "cot_fs_opt_train.jsonl.gz","cot_zs_noopt_train.jsonl.gz","cot_zs_opt_train.jsonl.gz"}

# load files that match the grep pattern
flanv2_subset = load_dataset("SirNeural/flan_v2", data_files=data_files)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [6]:
df = pd.DataFrame(flanv2_subset)

In [7]:
df["train"]

0         {'inputs': 'Q: Test for natural language infer...
1         {'inputs': 'Q: If "A man and a little girl are...
2         {'inputs': 'q: What do you do to someone easil...
3         {'inputs': 'QUESTION: People have to eat and d...
4         {'inputs': 'Test for natural language inferenc...
                                ...                        
373676    {'inputs': 'Janet takes two multivitamins and ...
373677    {'inputs': 'Steve finds 100 gold bars while vi...
373678    {'inputs': 'My question is: Fifteen more than ...
373679    {'inputs': 'Jack and Jill race up the hill to ...
373680    {'inputs': 'Lizzy: When all her clothes are wa...
Name: train, Length: 373681, dtype: object

In [10]:
df["train"].iloc[1]['task']

'cot'

In [11]:
df['train'] = df['train'].apply(lambda x: {k: v for k, v in x.items() if k != 'task'})

In [12]:
df["train"].iloc[1]

{'inputs': 'Q: If "A man and a little girl are sitting on a sidewalk near a blue bag eating." does that mean that "A father and daughter are sitting."? Options: - yes - it is not possible to tell - no A: A man and a little girl are not always father and daughter. The answer is it is not possible to tell. QUESTION: Given the sentence "A man uses a chainsaw to make an ice sculpture." can we conclude that "A man is making a sculpture for a wedding."? Options: - yes - it is not possible to tell - no Let\'s solve it slowly: Not all ice sculptures are made specifically for a wedding. Some are made for fun. The answer is it is not possible to tell. [QUESTION] Premise: "A pilot standing outside of an airplane." Hypothesis: "A pilot is about to go fly an airplane." Do we know that the hypothesis entailed by the premise? Being a pilot do not mean you fly a airplane 24 hours a day. The answer is it is not possible to tell. Q: Premise: "A dog swims holding a large stick in his mouth in front of a 

In [14]:
#df["train"].iloc[1]['task']

In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
def count_tokens(text):
    return len(tokenizer.tokenize(text))

In [20]:
print(count_tokens(df['train'].iloc[1]['targets']))

18


In [21]:
df_filtered = df[df['train'].apply(lambda x: count_tokens(x['targets']) >= 100)]

Token indices sequence length is longer than the specified maximum sequence length for this model (945 > 512). Running this sequence through the model will result in indexing errors


In [22]:
df_filtered.shape

(9122, 1)

In [25]:
df_filtered.loc[:,'combined'] = df_filtered['train'].apply(lambda x: x['inputs'] + ' ' + x['targets'])

In [26]:
print(df_filtered[['train', 'combined']].head())

                                                 train  \
34   {'inputs': 'Q: April went to a concert that ha...   
82   {'inputs': 'Macy's is selling shirts that have...   
87   {'inputs': 'Question: Stella collects stamps. ...   
136  {'inputs': 'Question: Jerry needs to shingle 3...   
156  {'inputs': 'Q: When Spencer jumps rope, he jum...   

                                              combined  
34   Q: April went to a concert that has two bands....  
82   Macy's is selling shirts that have been reduce...  
87   Question: Stella collects stamps. She counted ...  
136  Question: Jerry needs to shingle 3 roofs. Each...  
156  Q: When Spencer jumps rope, he jumps 4 times p...  


In [27]:
df_filtered['combined'].iloc[1]

"Macy's is selling shirts that have been reduced to $6. This price is at 25% of the original price. What was the original price? Give the rationale and then the answer. Let's think step by step. The original price is x. The discount is 100% - 25% remaining = 75% discount. The discount is 75%, so the original price is x - .75x = .25x. The sale price is $6, so $6 = .25x, which is the same as 6 / .25 = x. X = $24.. The answer is: 24. Marky owns a computer accessory shop. For this month, half of their sales are wireless mice, one-fourth are optical mice, and the rest are trackball mice. If Marky's shop was able to sell a total of 80 mice, how many of them are trackball mice? Give the rationale and then the answer. Let's think step by step. There were 80 / 2 = 40 wireless mice sold. There were 80 / 4 = 20 optical mice sold. So, there were a total of 40 + 20 = 60 wireless and optical mice sold. Therefore, 80 - 60 = 20 trackball mice were sold this month.. The answer is: 20. There is a very l

In [28]:
vectorizer = TfidfVectorizer().fit_transform(df_filtered['combined'])
vectors = vectorizer.toarray()

In [29]:
cosine_sim_matrix = cosine_similarity(vectors)

In [30]:
threshold = 0.95
similar_pairs = np.argwhere(cosine_sim_matrix > threshold)
to_remove = set()

for i, j in similar_pairs:
    if i != j and i not in to_remove and j not in to_remove:
        to_remove.add(j)  # remove the duplicate

In [31]:
indices_to_keep = [i for i in range(len(df_filtered)) if i not in to_remove]
deduplicated_df = df_filtered.iloc[indices_to_keep]

In [32]:
deduplicated_df.shape

(7309, 2)

In [33]:
deduplicated_df.head()

,train,combined
34,{'inputs': 'Q: April went to a concert that ha...,Q: April went to a concert that has two bands....
82,{'inputs': 'Macy's is selling shirts that have...,Macy's is selling shirts that have been reduce...
87,{'inputs': 'Question: Stella collects stamps. ...,Question: Stella collects stamps. She counted ...
136,{'inputs': 'Question: Jerry needs to shingle 3...,Question: Jerry needs to shingle 3 roofs. Each...
156,"{'inputs': 'Q: When Spencer jumps rope, he jum...","Q: When Spencer jumps rope, he jumps 4 times p..."


In [35]:
deduplicated_df.drop(columns=['combined'])


KeyError: "['combined'] not found in axis"

In [36]:
deduplicated_df.head()

,train
34,{'inputs': 'Q: April went to a concert that ha...
82,{'inputs': 'Macy's is selling shirts that have...
87,{'inputs': 'Question: Stella collects stamps. ...
136,{'inputs': 'Question: Jerry needs to shingle 3...
156,"{'inputs': 'Q: When Spencer jumps rope, he jum..."


In [37]:
deduplicated_df['train'].iloc[1]

{'inputs': "Macy's is selling shirts that have been reduced to $6. This price is at 25% of the original price. What was the original price? Give the rationale and then the answer. Let's think step by step. The original price is x. The discount is 100% - 25% remaining = 75% discount. The discount is 75%, so the original price is x - .75x = .25x. The sale price is $6, so $6 = .25x, which is the same as 6 / .25 = x. X = $24.. The answer is: 24. Marky owns a computer accessory shop. For this month, half of their sales are wireless mice, one-fourth are optical mice, and the rest are trackball mice. If Marky's shop was able to sell a total of 80 mice, how many of them are trackball mice? Give the rationale and then the answer. Let's think step by step. There were 80 / 2 = 40 wireless mice sold. There were 80 / 4 = 20 optical mice sold. So, there were a total of 40 + 20 = 60 wireless and optical mice sold. Therefore, 80 - 60 = 20 trackball mice were sold this month.. The answer is: 20. There 

In [38]:
deduplicated_df.to_csv('deduplicated_df.csv', index=False)